#Code used for my talk 'Cómo diseñar APIs en instantes usando Flask' at PyconES 2018
###(How to create APIs in no time with Flask)

To install the required packages:

* pip install Flask
* pip install Flask-SQLAlchemy


---



Let's create a 'Hello World' in Flask

In [0]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
  return 'hello world!'

if __name__ == '__main__':
  app.run(debug=True)

Let's return a Json

In [0]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def hello():
  return jsonify({'message' : 'hello world!'})

if __name__ == '__main__':
  app.run(debug=True)

###Let's build a Pizza API!

In [0]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')

def getPizza():
  pizza = {
      'name': 'hawaian',
      'image_url': 'http://example.com/hawaian_pizza.jpg'
  }
  return jsonify(pizza)

if __name__ == '__main__':
  app.run(debug=True)

We can also return a dictionary of pizzas to store multiple pizzas

In [0]:
from flask import Flask, jsonify

app = Flask(__name__)

pizza = [
    {
        'name': 'hawaian',
        'image_url': 'http://example.com/hawaian_pizza.jpg'
    },
    {
        'name': 'margerita',
        'image_url': 'http://example.com/margerita.jpg'
    }
]

#we'll now search by the dictionary index
@app.route('/<int:index>') 

def getPizza(index):
  
  return jsonify(pizza[index])

if __name__ == '__main__':
  app.run(debug=True)

But what if the index does not exist?

-> Return 404 error

In [0]:
from flask import Flask, jsonify, abort

app = Flask(__name__)

pizza = [
    {
        'name': 'hawaian',
        'image_url': 'http://example.com/hawaian_pizza.jpg'
    },
    {
        'name': 'margerita',
        'image_url': 'http://example.com/margerita.jpg'
    }
]

#we'll now search by the dictionary index
@app.route('/<int:index>') 

def getPizza(index):
  
  try:
    return jsonify(pizza[index])
  except IndexError:
    abort(404)

if __name__ == '__main__':
  app.run(debug=True)

Why search by the index while we can search the pizza by its name?

In [0]:
from flask import Flask, jsonify, abort

app = Flask(__name__)

pizza = [
    'hawaian': {
        'name': 'hawaian',
        'image_url': 'http://example.com/hawaian_pizza.jpg'
    },
    'margerita': {
        'name': 'margerita',
        'image_url': 'http://example.com/margerita.jpg'
    }
]

#we'll now search by the dictionary index
@app.route('/<slug>') 

def getPizza(slug):
  
  try:
    return jsonify(pizza[slug])
  except IndexError:
    abort(404)

if __name__ == '__main__':
  app.run(debug=True)

###Fetch from database

Let's define our db model

In [0]:
from flask_sqlalchemy import flask_sqlalchemy

db = SQLAlchemy()

class pizza(db.Model):
  id = db.Column(db.Integer, primary_key=True)
  slug = db.Column(db.String(30), index=True)
  name = db.Column(db.String(30), nullable=False)
  image_url = db.Column(db.String(150), nullable=False)

Let's save the pizzas to the db

In [0]:
from flask import Flask, jsonify
from models import db, Pizza
import sys

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///pizza.db'
db.init_app(app)

@app.route('/<slug>')
def getPizza(slug):
  pizza = Pizza.query.filter(Pizza.slug==slug).first_or_404()
  my_pizza = {
      'name': pizza.name,
      'image_url': pizza.image_url
  }
  return jsonify(my_pizza)

if __name__ == '__main__':
  if 'createdb' in sys.argv:
    with app.app_context():
      db.create_all()
    print('Database created')
  elif 'seeddb' in sys.argv:
    with app.app.context():
      pizza1 = Pizza(slug='hawaian', 
                     name='hawaian', 
                     image_url='http://example.com/hawaian.jpg')
      db.session.add(pizza1)
      pizza2 = Pizza(slug='margerita',
                     name='margerita',
                     image_url='http://example.com/margerita.jpg')
      db.session.add(pizza2)
      db.session.commit()
    print('saved to database')
  else:
    app.run(debug=True)

In [0]:
#To run it from the terminal:
python api_file.py createdb seeddb

Let's save the pizza by POST

In [0]:
@app.route('/', methods=['POST'])
def create_pizza():
  #validate
  name = request.form.get('name')
  if not name:
    return 'parameter required', 400
  image_url = request.form.get('image_url')
  if not image_url:
    return 'parameter required', 400
  slug = slugify(name)
  
  #save in db
  pizza = Pizza(slug=slug, name=name, image_url=image_url)
  db.session.add(pizza)
  db.session.commit(pizza)
  
  #response
  res = jsonify({'message':'pizza sucessfully created'})
  res.status_code = 201
  location = url_for('get_pizza', slug=slug)
  resp.headers['location'] = location
  return res
  